In [ ]:
!pip install datasets openpyxl transformers torch tiktoken verovio torchvision

In [ ]:
# !pip3 install torchvision

In [ ]:
!unzip /content/License-Characters-by-2-27classes.zip -d /content/Dataset/

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch

import random
import string
import math
import itertools
import os

import numpy as np
import imgaug
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.model_selection

from tensorflow.keras.models import load_model
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import xml.etree.ElementTree as ET

import cv2

from PIL import Image

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from transformers import AutoModel, AutoTokenizer, pipeline


In [4]:
tokenizer = AutoTokenizer.from_pretrained("stepfun-ai/GOT-OCR2_0", trust_remote_code=True)
model = AutoModel.from_pretrained("stepfun-ai/GOT-OCR2_0", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/stepfun-ai/GOT-OCR2_0:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/stepfun-ai/GOT-OCR2_0:
- render_tools.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


got_vision_b.py:   0%|          | 0.00/16.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/stepfun-ai/GOT-OCR2_0:
- got_vision_b.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/stepfun-ai/GOT-OCR2_0:
- modeling_GOT.py
- render_tools.py
- got_vision_b.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [5]:
xml_dir = '/content/Dataset/License-Characters-by-2-27classes/train'
image_dir = '/content/Dataset/License-Characters-by-2-27classes/train'

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text

    labels = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        labels.append(label)

    return filename, labels

def has_three_alphabets(labels):
    alphabet_count = sum([1 for label in labels if label.isalpha()])
    return alphabet_count == 3

image_labels = []
for xml_filename in os.listdir(xml_dir):
    if xml_filename.endswith('.xml'):
        xml_file_path = os.path.join(xml_dir, xml_filename)

        image_filename, labels = parse_xml(xml_file_path)

        if has_three_alphabets(labels):
            image_path = os.path.join(image_dir, image_filename)
            image_labels.append((image_path, labels))

In [6]:
image_paths = [x[0] for x in image_labels]
# Join the character lists into strings
labels = [''.join(x[1]) for x in image_labels]

# Tokenize the labels to ensure uniform length (max length 128 here)
# tokenized_labels = [tokenizer.encode(label, padding='max_length', truncation=True, max_length=128) for label in labels]


In [42]:
from datasets import Dataset
import os

# Assume image_paths is a list of paths and labels is a list of labels (same length)
data = {'image_path': image_paths, 'label': labels}

# Create Hugging Face Dataset
hf_dataset = Dataset.from_dict(data)

In [43]:
from transformers import TrOCRProcessor
from PIL import Image

# Initialize the processor for microsoft/trocr-base-stage1
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-stage1')

def preprocess_images(examples):
    # Convert image paths to images
    images = [Image.open(path).convert("RGB") for path in examples['image_path']]

    # Preprocess images (resize, normalize)
    pixel_values = processor(images, return_tensors="pt", padding=True).pixel_values

    # Tokenize the text labels (license plate strings)
    labels = processor.tokenizer(examples['label'], return_tensors="pt", padding=True).input_ids

    # Return both pixel values and tokenized labels
    return {'pixel_values': pixel_values, 'labels': labels}

# Apply the preprocessing to the dataset
hf_dataset = hf_dataset.map(preprocess_images, batched=True)


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [45]:
# Split the dataset into training (80%) and validation (20%)
train_test_split = hf_dataset.train_test_split(test_size=0.2)

# Create a dictionary for train and validation datasets
hf_dataset = {
    'train': train_test_split['train'],
    'validation': train_test_split['test']
}

In [37]:
from transformers import VisionEncoderDecoderModel

# Load the pre-trained model for OCR
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-stage1')

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",     # Evaluate every epoch
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=5,              # Number of epochs
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
    save_steps=500,                  # Save the model every 500 steps
    save_total_limit=2,              # Limit the number of saved model checkpoints
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset['train'],
    eval_dataset=hf_dataset['validation'],
    tokenizer=processor,  # Use the processor directly
)

# Train the model
trainer.train()


ValueError: too many dimensions 'str'

In [40]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset['train'],
    eval_dataset=hf_dataset['validation'],
    tokenizer=processor.feature_extractor,  # Use processor's feature extractor
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/models/trocr/processing_trocr.py:137: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


ValueError: too many dimensions 'str'

In [ ]:
# Evaluate the model on the validation set
metrics = trainer.evaluate(eval_dataset=hf_dataset['validation'])
# print(metrics)

# Save the model and processor for future use
model.save_pretrained("./fine_tuned_trocr_model")
processor.save_pretrained("./fine_tuned_trocr_processor")


In [9]:
# labels[0]

In [10]:
# tokenized_label = []
# for i in labels:
#   tokenized_label.append(tokenizer.encode_plus(
#           i,                 # Pass the label string directly
#           padding='max_length',  # Ensure uniform length
#           truncation=True,       # Truncate if longer than max_length
#           max_length=128,        # Adjust max length as per your requirement
#           return_tensors='tf'    # Return as TensorFlow tensor
#       )['input_ids'][0])  # We get the first element as the tensor

In [ ]:
# import tensorflow as tf

# # Preprocess function to handle image resizing
# def preprocess_image(image_path):
#     """
#     Preprocess image only.
#     Args:
#         image_path: Path to the image file.

#     Returns:
#         image: Preprocessed image tensor.
#     """
#     # Load and resize the image
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_jpeg(image, channels=3)

#     # Resize the image to 224x224 (or your model's expected size)
#     image = tf.image.resize(image, (224, 224))

#     # Normalize the image pixel values to the range [0, 1]
#     image = image / 255.0
#     return image

# # Create a TensorFlow dataset
# # dataset = tf.data.Dataset.from_tensor_slices((image_paths, tokenized_label))

# # Step 1: Process the images first and keep the original label intact
# dataset = dataset.map(lambda img, lbl: (preprocess_image(img), lbl))

# # Batch the dataset for training
# dataset = dataset.batch(2)

# # Print the output of a batch to verify
# for batch in dataset.take(1):
#     print(batch)


(<tf.Tensor: shape=(2, 224, 224, 3), dtype=float32, numpy=
array([[[[0.18431373, 0.13725491, 0.13725491],
         [0.18679972, 0.1409839 , 0.13725491],
         [0.19107142, 0.14739147, 0.13725491],
         ...,
         [1.        , 0.9702557 , 0.9843137 ],
         [1.        , 0.9766632 , 0.9843137 ],
         [1.        , 0.98039216, 0.9843137 ]],

        [[0.18461135, 0.13755253, 0.13755253],
         [0.18700302, 0.14118718, 0.13745819],
         [0.19111261, 0.14743266, 0.13729608],
         ...,
         [1.        , 0.9707685 , 0.98457015],
         [1.        , 0.9768518 , 0.984408  ],
         [1.        , 0.98039216, 0.9843137 ]],

        [[0.18611693, 0.13905811, 0.13905811],
         [0.18803138, 0.14221556, 0.13848656],
         [0.19132099, 0.147641  , 0.13750446],
         ...,
         [1.        , 0.973363  , 0.9858674 ],
         [1.        , 0.9778063 , 0.9848852 ],
         [1.        , 0.98039216, 0.9843137 ]],

        ...,

        [[0.2025737 , 0.17755617,

In [11]:
# import torch
# from torch.optim import Adam

# optimizer = Adam(model.parameters(), lr=5e-5)

# criterion = torch.nn.CrossEntropyLoss()

# def train(model, tokenizer, dataset, epochs=3):
#     """
#     Train the OCR model.

#     Args:
#         model: The OCR model.
#         tokenizer: Tokenizer corresponding to the model.
#         dataset: A dataset of (image_file, label) pairs.
#         epochs: Number of epochs for training.

#     Returns:
#         None
#     """
#     model.train()  # Set model to training mode

#     for epoch in range(epochs):
#         total_loss = 0
#         for image_file, label in dataset:  # Loop over the dataset
#             # Use the model's chat method to perform OCR on the image
#             pred_text = model.chat(tokenizer, image_file, ocr_type='ocr')

#             # Tokenize the predicted text and ground truth label for comparison
#             pred_tokens = tokenizer(pred_text, return_tensors='pt', padding=True, truncation=True)['input_ids']
#             label_tokens = tokenizer(label, return_tensors='pt', padding=True, truncation=True)['input_ids']

#             # Ensure tensors are the same length for loss calculation
#             if pred_tokens.shape != label_tokens.shape:
#                 min_len = min(pred_tokens.shape[1], label_tokens.shape[1])
#                 pred_tokens = pred_tokens[:, :min_len]
#                 label_tokens = label_tokens[:, :min_len]

#             # Compute the loss
#             loss = criterion(pred_tokens.squeeze(0), label_tokens.squeeze(0))

#             # Zero out previous gradients
#             optimizer.zero_grad()

#             # Backpropagate the loss
#             loss.backward()

#             # Update the model parameters
#             optimizer.step()

#             # Accumulate the loss for reporting
#             total_loss += loss.item()

#         # Print the average loss per epoch
#         print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataset)}")


# # Start training
# train(model, tokenizer, dataset, epochs=3)

In [12]:
# def preprocess_function(examples):
#     images = []
#     labels = []

#     for image_path, label_chars in examples:
#         image = Image.open(image_path).convert("RGB")
#         images.append(image)

#         label_text = ''.join(label_chars)

#         labels.append(tokenizer(label_text, padding="max_length", truncation=True, return_tensors="pt"))

#     return {"images": images, "labels": labels}

# # processed_dataset = preprocess_function(image_labels)